### Exercise 1. Smith-Waterman

In [1]:
import numpy as np
import scipy as sc

In [2]:
def match(a,b, mu):
    if a == b:
        return 3
    else:
        return mu

In [17]:
def smith_water(str1, str2, mu, gap, match:callable):
    #Computing matrix
    
    str1 = str1.upper()
    str2 = str2.upper()
    l1 = len(str1)
    l2 = len(str2)
    matrix = np.zeros((l1+1, l2+1))
    #k=1
    
    for i in range(1, l1+1):
        matrix[i][0] = 0
    
    for i in range(1, l2+1):
        matrix[0][i] = 0
    
    for i in range(1, l2+1):
        for j in range(1, l1+1):
            F_1 = matrix[j-1][i-1] + match(str1[j-1], str2[i-1], mu)
            F_2 = matrix[j-1][i] + gap
            F_3 = matrix[j][i-1] + gap
            matrix[j, i] = max(F_1, F_2, F_3, 0)
    
    #As matrix computed, we reach returning point
    
    algn1 = ''
    algn2 = ''
    arg_max = np.where(matrix == np.amax(matrix))
    a, b = arg_max[0][0], arg_max[1][0]
    while matrix[a][b] != 0 :
        '''if matrix[a][b] == matrix[a-1][b-1] + match(str1[a-1], str2[b-1], mu):
            algn1 += str1[a-1]
            algn2 += str2[b-1]
            a = a-1
            b = b-1
        elif matrix[a][b] == matrix[a-1][b] + gap:
            algn1 += str1[a-1]
            algn2 += '-'
            a = a-1
        else:
            algn1 += '-'
            algn2 += str2[b-1]
            b = b-1'''
        max_res = max(matrix[a-1][b-1], matrix[a][b-1], matrix[a-1][b])
        if max_res == matrix[a-1][b-1]:
            algn1 += str1[a-1]
            algn2 += str2[b-1]
            a = a-1
            b = b-1
        elif max_res == matrix[a-1][b]:
            algn1 += str1[a-1]
            algn2 += '-'
            a = a-1
        else:
            algn1 += '-'
            algn2 += str2[b-1]
            b = b-1
    '''if a>0:
        while a != 0:
            algn1 += str1[a-1]
            algn2 += '-'
            a = a-1
    if b>0:
        while b != 0:
            algn1 += '-'
            algn2 += str2[b-1]
            b = b-1'''
    
    # Backtracing
    
    algn1 = algn1[::-1]
    algn2 = algn2[::-1]
    
    algn1_fin = str('')
    algn2_fin = str('')
    
    str1_rev = str1[::-1]
    str2_rev = str2[::-1]
    
    hype_1_1 = algn1.replace('-','')
    hype_1_2 = algn1[::-1].replace('-','')
    
    index_1_1 = str1.find(hype_1_1)
    index_1_2 = str1_rev.find(hype_1_2)
    
    hype_2_1 = algn2.replace('-','')
    hype_2_2 = algn2[::-1].replace('-','')
    
    index_2_1 = str2.find(hype_2_1)
    index_2_2 = str2_rev.find(hype_2_2)
    
    if index_2_1 > index_1_1:
        algn1_fin = '-' * (index_2_1-index_1_1)
    else:
        algn2_fin = '-' * (index_1_1-index_2_1)        
    
    algn1_fin += str1[:index_1_1] + algn1 + str1_rev[index_1_2-1:0:-1]+str1_rev[0]
    
    algn2_fin += str2[:index_2_1] + algn2 + str2_rev[index_2_2-1:0:-1]+str2_rev[0]
    
    return algn1_fin, algn2_fin, matrix

In [18]:
#Запустил с gap = -0.4, чтобы были видны различия
a1, a2, matrix = smith_water('SSTRARSS', 'HASMDTRARKKSS', -3, -1, match)
print(a1)
print(a2)

---SSTRARSS
HASMDTRARKKSS


In [20]:
#Запустил с gap = -0.7, чтобы были видны различия
a1, a2, matrix = smith_water('SSTRARSS', 'HASMDTRARKKSS', -1, -1, match)
print(a1)
print(a2)

---SSTRARSS
HASMDTRARKKSS


#### Exercise 2: Aphine gap

In [132]:
# Creatnig all functions

global infinity

infinity = -float('inf')

#return match or mismatch score
def match_greek(a, b, match, mismatch):
    if a == b:
        return match
    else:
        return mismatch

#initializers for matrices
def init_side(i, j, sigma, init_gap):
    if i > 0 and j == 0:
        return infinity
    else:
        if j > 0:
            return init_gap + (sigma * j)
        else:
            return 0

def init_down(i, j, sigma, init_gap):
    if j > 0 and i == 0:
        return infinity
    else:
        if i > 0:
            return init_gap + (sigma * j)
        else:
            return 0

def init_main(i, j):
    if j == 0 and i == 0:
        return 0
    else:
        if j == 0 or i == 0:
            return infinity
        else:
            return 0

def greek_crap(str1, str2, match, mismatch, init_gap, sigma):
    '''Finding affine_gap alligning of two strings'''
    
    str1 = str1.upper()
    str2 = str2.upper()
    
    param_1 = len(str2) + 1
    param_2 = len(str1) + 1
    
    # Making matrixes
    X = [[init_side(i, j, sigma, init_gap) for j in range(0, param_2)] for i in range(0, param_1)]
    Y = [[init_down(i, j, sigma, init_gap) for j in range(0, param_2)] for i in range(0, param_1)]
    M = [[init_main(i, j) for j in range(0, param_2)] for i in range(0, param_1)]

    # Computing matrixes
    for j in range(1, param_2):
        for i in range(1, param_1):
            X[i][j] = max((init_gap + sigma + M[i][j-1]), (sigma + X[i][j-1]), (init_gap + sigma + Y[i][j-1]))
            Y[i][j] = max((init_gap + sigma + M[i-1][j]), (init_gap + sigma + X[i-1][j]), (sigma + Y[i-1][j]))
            M[i][j] = max(match_greek(str1[j-1], str2[i-1], match, mismatch) + M[i-1][j-1], X[i][j], Y[i][j])
    
    #Backtracing
    allgn1 = ''
    allgn2 = ''

    i = len(str2)
    j = len(str1)
    while (i > 0 or j > 0):
        #print(M[i][j], Y[i][j])
        print(i,j)
        if (i > 0 and j > 0 and M[i][j] == M[i-1][j-1] + match_greek(str1[j-1], str2[i-1], match, mismatch)):
            allgn1 += str1[j-1]
            allgn2 += str2[i-1]
            i -= 1
            j -= 1
            #print(i,j)
        elif (i > 0 and M[i][j] == Y[i][j]):
            allgn1 += '-'
            allgn2 += str2[i-1]
            i -= 1
            print(i,j)
        elif (j > 0 and M[i][j] == X[i][j]):
            allgn1 += str1[j-1]
            allgn2 += '-'
            j -= 1
            #print(i,j)

    allgn1_fin = allgn1[::-1]
    allgn2_fin = allgn2[::-1]

    return allgn1_fin, allgn2_fin

In [133]:
a1, a2 = greek_crap('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC','AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', match = 1., mismatch = -1, init_gap = 0, sigma   = -1)

#M, X, Y = greek_crap('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC','AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', match = 1., mismatch = -1, init_gap = 0, sigma   = -1)

print(a1)
print(a2)

36 36
35 35
34 34
34 33
33 32
32 31
31 30
30 29
29 29
29 29
28 28
27 27
26 26
25 25
24 24
24 23
23 22
23 21
22 20
21 19
21 18
20 17
20 16
20 15
20 14
19 13
18 12
17 11
17 10
16 9
16 8
15 7
14 6
13 5
13 4
12 3
11 3
11 3
10 3
10 3
9 2
8 1
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0

8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


...

8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0
8 0


KeyboardInterrupt: 

In [131]:
from pprint import pprint

global S
global E
global match
global mismatch
global MIN
S   =  0
E   =  -1
match = 1.
mismatch = -1.
MIN = -float("inf")

#return match or mismatch score
def _match(s, t, i, j):
    if t[i-1] == s[j-1]:
        return match
    else:
        return mismatch

#initializers for matrices
def _init_x(i, j):
    if i > 0 and j == 0:
        return MIN
    else:
        if j > 0:
            return -10 + (-0.5 * j)
        else:
            return 0

def _init_y(i, j):
    if j > 0 and i == 0:
        return MIN
    else:
        if i > 0:
            return -10 + (-0.5 * i)
        else:
            return 0

def _init_m(i, j):
    if j == 0 and i == 0:
        return 0
    else:
        if j == 0 or i == 0:
            return MIN
        else:
            return 0

def _format_tuple(inlist, i, j):
    return 0

def distance_matrix(s, t):
    dim_i = len(t) + 1
    dim_j = len(s) + 1
    #abuse list comprehensions to create matrices
    X = [[_init_x(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]
    Y = [[_init_y(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]
    M = [[_init_m(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]

    for j in range(1, dim_j):
        for i in range(1, dim_i):
            X[i][j] = max((S + E + M[i][j-1]), (E + X[i][j-1]), (S + E + Y[i][j-1]))
            Y[i][j] = max((S + E + M[i-1][j]), (S + E + X[i-1][j]), (E + Y[i-1][j]))
            M[i][j] = max(_match(s, t, i, j) + M[i-1][j-1], X[i][j], Y[i][j])

    return X, Y, M

def backtrace(s, t, X, Y, M):
    sequ1 = ''
    sequ2 = ''
    i = len(t)
    j = len(s)
    while (i>0 or j>0):
        print(i,j)
        if (i>0 and j>0 and M[i][j] == M[i-1][j-1] + _match(s, t, i, j)):
            sequ1 += s[j-1]
            sequ2 += t[i-1]
            i -= 1; j -= 1
        elif (i>0 and M[i][j] == Y[i][j]):
            sequ1 += '_'
            sequ2 += t[i-1]
            i -= 1
        elif (j>0 and M[i][j] == X[i][j]):
            sequ1 += s[j-1]
            sequ2 += '_'
            j -= 1

    sequ1r = ' '.join([sequ1[j] for j in range(-1, -(len(sequ1)+1), -1)])
    sequ2r = ' '.join([sequ2[j] for j in range(-1, -(len(sequ2)+1), -1)])

    return sequ1r, sequ2r

X, Y, M = distance_matrix('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC','AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC')
a1, a2 = backtrace('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC','AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', X, Y, M)
print(a1)
print(a2)

36 36
35 35
34 34
34 33
33 32
32 31
31 30
30 29
29 29
28 28
27 27
26 26
25 25
24 24
24 23
23 22
23 21
22 20
21 19
21 18
20 17
20 16
20 15
20 14
19 13
18 12
17 11
16 10
15 9
15 8
14 7
13 7
12 7
11 6
10 5
9 4
8 3
7 3
6 2
5 1
4 1
3 1
2 1
1 1
T _ _ _ _ C C _ C A G T _ _ T A T G T C A G G G G A C A C G A G C A T G _ C A G A G A C
A A T T G C C G C C G T C G T _ T T T C A _ _ _ G _ C A _ G _ T T A T G T C A G A _ T C


In [ ]:
36 36
35 35
34 34
34 33
33 32
32 31
31 30
30 29
29 29
29 29
28 28
27 27
26 26
25 25
24 24
24 23
23 22
23 21
22 20
21 19
21 18
20 17
20 16
20 15
20 14
19 13
18 12
17 11
17 10
16 9
16 8
15 7
14 6
13 5
13 4
12 3
11 3
11 3
10 3
10 3